In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [8]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [10]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [11]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Metal device set to: Apple M1 Pro
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________

2022-12-05 21:03:49.629182: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-05 21:03:49.629726: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [14]:
epochs = 100
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/100
2/2 [==============================] - 0s 10ms/step - loss: 0.5255 - accuracy: 0.8182
Epoch 2/100
2/2 [==============================] - 0s 8ms/step - loss: 0.5232 - accuracy: 0.8182
Epoch 3/100
2/2 [==============================] - 0s 8ms/step - loss: 0.5207 - accuracy: 0.8182
Epoch 4/100
2/2 [==============================] - 0s 9ms/step - loss: 0.5189 - accuracy: 0.8182
Epoch 5/100
2/2 [==============================] - 0s 8ms/step - loss: 0.5176 - accuracy: 0.8182
Epoch 6/100
2/2 [==============================] - 0s 8ms/step - loss: 0.5167 - accuracy: 0.8182
Epoch 7/100
2/2 [==============================] - 0s 8ms/step - loss: 0.5169 - accuracy: 0.8182
Epoch 8/100
2/2 [==============================] - 0s 8ms/step - loss: 0.5145 - accuracy: 0.8182
Epoch 9/100
2/2 [==============================] - 0s 7ms/step - loss: 0.5112 - accuracy: 0.8182
Epoch 10/100
2/2 [==============================] - 0s 8ms/step - loss: 0.5098 - accuracy: 0.8182
Epoch 11/100
2/2 [==========

2/2 [==============================] - 0s 7ms/step - loss: 0.4212 - accuracy: 0.9091
Epoch 85/100
2/2 [==============================] - 0s 8ms/step - loss: 0.4196 - accuracy: 0.9091
Epoch 86/100
2/2 [==============================] - 0s 7ms/step - loss: 0.4174 - accuracy: 0.9091
Epoch 87/100
2/2 [==============================] - 0s 8ms/step - loss: 0.4152 - accuracy: 0.9091
Epoch 88/100
2/2 [==============================] - 0s 7ms/step - loss: 0.4136 - accuracy: 0.9091
Epoch 89/100
2/2 [==============================] - 0s 8ms/step - loss: 0.4112 - accuracy: 0.9091
Epoch 90/100
2/2 [==============================] - 0s 8ms/step - loss: 0.4090 - accuracy: 0.9091
Epoch 91/100
2/2 [==============================] - 0s 8ms/step - loss: 0.4067 - accuracy: 0.9091
Epoch 92/100
2/2 [==============================] - 0s 7ms/step - loss: 0.4056 - accuracy: 0.9091
Epoch 93/100
2/2 [==============================] - 0s 8ms/step - loss: 0.4050 - accuracy: 0.9091
Epoch 94/100
2/2 [===============

In [15]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets
